<a href="https://colab.research.google.com/github/HaiderAltaf/CS6910_Assignment1/blob/branch-1/cs6910_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Functions needed for construction of our neural network



In [16]:
def weights(weight_init, n, L, X_train, no_of_classes):
  
  Weights = []
  np.random.seed(0)

  if weight_init =="random":
    temp = np.random.rand(n, len(X_train[0]))
    Weights.append(temp)

    for i in range(1, L-1):
      temp = np.random.rand(n, n)
      Weights.append(temp)

    temp = np.random.rand(no_of_classes, n)
    Weights.append(temp)

  if weight_init =="xavier":
    scale = 1/max(1, (2+2)/2 )
    limit = math.sqrt(3*scale)

    temp = np.random.uniform(-limit, limit, size=(n, len(X_train[0]))) 
    Weights.append(temp)
    for i in range(1, L-1):
      temp  = np.random.uniform(-limit, limit, size=(n,n))
      Weights.append(temp)

    temp = np.random.uniform(-limit, limit, size=(no_of_classes, n)) 
    Weights.append(temp)

  if weight_init ==3:
    temp = np.zeros((n, len(X_train[0])))
    Weights.append(temp)

    for i in range(1, L-1):
      temp = np.zeros((n, n))
      Weights.append(temp)

    temp = np.zeros((no_of_classes, n))
    Weights.append(temp)

  
  return Weights



In [17]:
def biases(weight_init, n, L, y_train, no_of_classes):
  
  bias = []
  np.random.seed(0)

  if weight_init =="random":
    for i in range(L-1):
      temp = np.random.rand(n)  # for schochastic GD
      #temp = np.random.rand(len(y_train),n)  # for Vanilla GD
      bias.append(temp)
    
    temp = np.random.rand(no_of_classes)   # for schochastic GD
    #temp = np.random.rand(len(y_train), no_of_classes)   # for Vanilla GD
    bias.append(temp)

  if weight_init =="xavier":
    scale = 1/max(1, (2+2)/2 )
    limit = math.sqrt(3*scale)
    for i in range(L-1):
      temp  = np.random.uniform(-limit, limit, size=(n))   # for schochastic GD
      #temp  = np.random.uniform(-limit, limit, size=(len(y_train),n))  # for Vanilla GD
      bias.append(temp)

    temp = np.random.uniform(-limit, limit, size=(no_of_classes))   # for schochastic GD
    #temp = np.random.uniform(-limit, limit, size=(len(y_train),no_of_classes)) # for Vanilla GD
    bias.append(temp)

  if weight_init ==3:
    for i in range(L-1):
      temp = np.zeros(n)
      bias.append(temp)

    temp = np.zeros(no_of_classes)
    bias.append(temp)

  
  return bias


Activation Functions

In [18]:
def sigmoid(a):
  a = np.float128(a)
  return 1/(1+np.exp(-a))

def tanh(a):
  return (np.exp(a)-np.exp(-a))/(np.exp(a)+np.exp(-a))

def ReLu(a, weight_init):
  i=0
  for ele in a:
    a[i] = max(0,ele)
    i+=1
  if weight_init == "random":
    return a - max(a)
  
  return a

def softmax(a):
  # temp = np.zeros_like(a)
  # for i in range(len(temp)):
  #   temp[i] = np.float128(a[i])
  #exp_logits = np.exp(a)
  return np.exp(a)/np.sum(np.exp(a))   

def der_softmax(a):
  return softmax(a)*(1- softmax(a))
  
def der_sigmoid(a):
  return sigmoid(a)*(1-sigmoid(a))

def der_tanh(a):
  return 1-(tanh(a)*tanh(a))

def der_ReLu(a):

  # it will create a matrix of same dimension as of a.
  gradient = np.zeros_like(a)  
  # sets the entries of gradient to 1 where the corresponding entries of x>=0
  gradient[a >= 0] = 1
  return gradient

Loss functions

In [19]:
def cross_entropy_loss(y_dash, y_train, X_train):
  losses = -np.log(y_dash[y_train])/len(X_train)
  return losses

def MSE_loss(y_dash, y_train, X_train):
  y_train_modified = np.zeros(10)
  y_train_modified[y_train] = 1
  losses = (np.sum((y_dash - y_train_modified)**2))/len(X_train)
  return losses

# Question 1
Download the fashion-MNIST dataset and plot 1 sample image for each class as shown in the grid below. Use from keras.datasets import fashion_mnist for getting the fashion mnist dataset. Show each sample class in wandb

In [20]:
import numpy as np
import pandas as pd
from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
import math

In [21]:
!pip install wandb
import wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=c1b503c4f2738a64239474ae411d6950dcabd4be4a2beedd8bdc3409a1f603be
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools


In [22]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
X_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2] )/255
X_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])/255

4422102/4422102 [==============================] - 0s 0us/step


In [23]:
validation_size = int(len(X_train)*0.1)

# randomly shuffle the indices of the data
shuffled_indices = np.random.permutation(len(X_train))

# split the shuffled data into training and validation sets
train_indices, validation_indices = shuffled_indices[:-validation_size], shuffled_indices[-validation_size:]
X_train, X_validation = X_train[train_indices], X_train[validation_indices]
y_train, y_validation = y_train[train_indices], y_train[validation_indices]

In [32]:
hid_layer = int(input("Enter the number of Hidden + outer layer: "))

Enter the number of Hidden + outer layer: 3


In [33]:
no_of_neuron = int(input("Enter the numbers of neuron in each hidden layer: "))

Enter the numbers of neuron in each hidden layer: 32


In [34]:
no_of_classes = len(np.unique(y_train))

In [35]:
weight_init = input("For random weights initialisation enter random and for xavier enter xavier: ")

For random weights initialisation enter random and for xavier enter xavier: random


In [12]:
wandb.init(entity= "am22s020", project="cs6910_trial")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: am22s020. Use `wandb login --relogin` to force relogin


In [ ]:
def plot_class_sample():
  class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
                'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
            
  no_of_classes = len(class_names)

  list_of_images  = []   # to give to the wandb

  for i in range(no_of_classes):
    
      # Find the index of the first image of each class
      idx = np.where(y_train == i)[0][0]
      
      # Plot the image
      image = X_train[idx].reshape(28,28)
      list_of_images.append((image, class_names[i]))

  # Plot the images in a grid
  fig, axes = plt.subplots(1, no_of_classes, figsize=(12,5))
  for i in range(no_of_classes):
      image, label = list_of_images[i]
      axes[i].imshow(image, cmap='gray')
      axes[i].set_title(label)
      axes[i].axis('off')

  wandb.log({"Question 1": [wandb.Image(img, caption=caption) for img, caption in list_of_images]})


# Question 2 

Implement a feedforward neural network which takes images from the fashion-mnist data as input and outputs a probability distribution over the 10 classes.

In [24]:
def forward_propagation(Weights, bias, x_input, hid_layer, acti_fun):
  
  L = hid_layer
  h = x_input
  a_out = []
  h_out = []
  h_out.append(h)
  
  ## for hidden layers
  for k in range(L-1):
    a = np.matmul(Weights[k], h) + bias[k]
    a_out.append(a)
    ## default activation function is sigmoid 
    if acti_fun == 'sigmoid':
      h = sigmoid(a)
    elif acti_fun == 'ReLu':
      h = ReLu(a)
    elif acti_fun == 'tanh':
      h = tanh(a)
    h_out.append(h)

  ## In outer layer softmax function
  a = np.matmul(Weights[L-1], h) + bias[L-1]
  a_out.append(a)
  y_dash = softmax(a)
  

  return a_out, h_out, y_dash


# Question-3
Implement the backpropagation algorithm with support for the following optimisation functions

    sgd
    momentum based gradient descent
    nesterov accelerated gradient descent
    rmsprop
    adam
    nadam 


Backward propagation

In [59]:
Weights = weights('random', 32, 3, X_train, 10)

In [60]:
bias = biases('random', 32, 3, y_train, 10)

In [79]:
def backward_propagation(a_out, h_out, y_train, y_dash, Weights,
                         hid_layer, loss_fu, acti_fun, L2_decay):
  
  L = hid_layer
  grad_W = [0]*L
  grad_b = [0]*L

  
  ## change each y_train into an array of 10 values
  y_train_modified = np.zeros(10)
  y_train_modified[y_train] = 1
    
  if loss_fu =='cross_entropy':
    output_gradient = -(y_train_modified - y_dash) #+ L2_decay*np.sum(Weights[0]) + L2_decay*np.sum(Weights[1])+ L2_decay*np.sum(Weights[2])
  else:
    output_gradient = (y_dash-y_train_modified )*der_softmax(a_out[L-1]) # + L2_decay*np.sum(Weights[0]) + L2_decay*np.sum(Weights[1])+ L2_decay*np.sum(Weights[2])

  for k in range(L, 0, -1):

    ## compute gradients w.r.t parameters
    W_gradient = np.matmul(output_gradient.reshape(len(output_gradient),1), 
                           h_out[k-1].reshape(1,len(h_out[k-1]))) 
    grad_W[k-1] = W_gradient

    b_gradients = output_gradient 
    grad_b[k-1] = b_gradients
   
    if k==1:
      continue
    ## compute gradients w.r.t layer below
    weight = Weights[k-1]
    h_gradient = np.matmul(weight.T, output_gradient)

    ## compute the gradient of pre activation layer
    if acti_fun == 'sigmoid':
      output_gradient = np.multiply(h_gradient, der_sigmoid(a_out[k-2]))
    elif acti_fun == 'ReLu':
      output_gradient = np.multiply(h_gradient, der_ReLu(a_out[k-2]))
    elif acti_fun == 'tanh':
     output_gradient = np.multiply(h_gradient, der_tanh(a_out[k-2]))
    

  return grad_W, grad_b



In [51]:
np.sum(Weights[0])+ np.zeros(5)+np.sum(Weights[1])+np.sum(Weights[2])

array([13139.59580177, 13139.59580177, 13139.59580177, 13139.59580177,
       13139.59580177])

Model Loss

In [26]:
def model_loss(X, Y, Weights, bias, hid_layer, loss_fu,
               L2_decay, X_train, X_validation, acti_fun):
  
  L = hid_layer
  loss = 0
 # i = 0
  for x,y in zip(X, Y):
      _, _, y_dash = forward_propagation(Weights, bias, x, L, acti_fun)
      #i+=1
      
      if loss_fu == 'cross_entropy':
        loss_iter = cross_entropy_loss(y_dash, y, X)
      elif loss_fu == 'mse':
        loss_iter = MSE_loss(y_dash, y_train, X)
          
      loss += loss_iter

      # if i==len(X_validation):
      #   break


  # Adding L2 regularization loss after an epochS
  for i in range(len(Weights)):
    loss = loss + 2*L2_decay*np.sum(Weights[i]**2)/len(X)

  return loss

  


In [ ]:
model_loss(X_test, y_test, Weights, bias, hid_layer, 'cross_entropy', 0.5, X_train, X_validation, acti_fun)

1.2371012814240635754

Model Accuracy

In [27]:
def Model_accuracy(X, Y, Weights, bias, hid_layer, acti_fun):
  
  L = hid_layer
  y_pred = np.zeros((len(X), 10))
  i=0
  for x in X:
    _, _, y_dash = forward_propagation(Weights, bias, x, L, acti_fun)
    y_pred[i] = y_dash
    i+=1

  correct = 0
  for array,y in zip(y_pred, Y):
    if np.argmax(array)==y:
      correct+=1
  accuracy = correct*100/len(X)

  return  accuracy

In [ ]:
Model_accuracy(X_validation, y_validation, Weights, bias, hid_layer, acti_fun)

71.3

#Mini Batch Gradient Descent 
if batch_size = 1, the algorithm will be stochastic gradient descent.
if batch_size = Number of samples, the algorithm will be vanilla gradient descent


In [77]:
def gradient_descent(learning_rate, Weights, bias, hid_layer, no_of_neuron,
                     y_train, X_train, batch_size, L2_decay, loss_fu, acti_fun):
  
  no_of_classes = len(np.unique(y_train))
  L = hid_layer
  n = no_of_neuron
  ## initialize the gradients of weights and biases
  dW = weights(3, n, L, X_train, no_of_classes)
  dB = biases(3, n, L, y_train, no_of_classes)
  ## initialize the loss
  loss = 0
  num_points_seen = 0

  for x,y in zip(X_train,y_train):

    ##x,y = np.float128(x), np.float128(y)

    ## Forward propagation
    a_out, h_out, y_dash = forward_propagation(Weights, bias, x, L, acti_fun)

    ## updating loss after each sample(not using this)
    
    # if loss_fu == 'cross_entropy':
    #   loss_iter = cross_entropy_loss(y_dash, y, X_train)  # for cross entropy loss 
    # else:
    #   loss_iter = MSE_loss(y_dash, y_train, X_train)  # for MSE loss

    # loss += loss_iter


    ## Backward Propagation
    grad_W, grad_b = backward_propagation(a_out, h_out, y_train, y_dash, Weights,
                                          hid_layer, loss_fu, acti_fun, L2_decay)

    ## Adding the gradients of weights and biases
    dW = [dW[i] + grad_W[i] for i in range(len(dW))]
    dB = [dB[i] + grad_b[i] for i in range(len(dB))]
    
    num_points_seen+=1
    
    if num_points_seen%batch_size == 0:

      ## Add L2 regularization penalty to gradient
      dW = [dW[i] + L2_decay*Weights[i] for i in range(len(dW))]
      # Weights updates
      
      Weights = [Weights[i] - dW[i]*learning_rate for i in range(L)]

      # Biases updates
      bias = [bias[i] - dB[i]*learning_rate for i in range(L)]

      
      ## initialize the gradients of weights and biases
      dW = weights(3, n, L, X_train, no_of_classes)
      dB = biases(3, n, L, y_train, no_of_classes)

  # Training loss of an epoch
  loss  = model_loss(X_train, y_train, Weights, bias, hid_layer, loss_fu,
                     L2_decay, X_train, X_validation, acti_fun)
  # Adding L2 regularization loss after an epochS
  for i in range(len(Weights)):
    loss = loss + 0.5*L2_decay*np.sum(Weights[i]**2)/len(X_train)
    

  #print(epoch, loss)

  return Weights, bias, loss


In [ ]:
for i in range(2):
  Weights, bias, loss = gradient_descent(0.01, Weights, bias, hid_layer, no_of_neuron,
                                         y_train, X_train, 25, 0.5,'cross_entropy', 'sigmoid')
  print(i, loss)
  val_accuracy  = Model_accuracy(X_train, y_train, Weights, bias, hid_layer, 'sigmoid')
  test_accuracy = Model_accuracy(X_test, y_test, Weights, bias, hid_layer, 'sigmoid')
  val_loss      = model_loss(X_train, y_train, Weights, bias, hid_layer, 'cross_entropy', 0, X_train, X_validation, 'sigmoid')
  test_loss     = model_loss(X_test, y_test, Weights, bias, hid_layer, 'cross_entropy', 0, X_train, X_validation, 'sigmoid')
  print('val_accuracy',val_accuracy,'test_accuracy',test_accuracy,'val_loss', val_loss,'test_loss',test_loss)
# Finish the WandB run
#wandb.finish()

In [ ]:
Model_accuracy(X_validation, y_validation, Weights, bias, hid_layer, 'sigmoid')

84.66666666666667

#Mini Batch Momentum based Gradient Descent
if batch_size = 1, the algorithm will be stochastic gradient descent. if batch_size = Number of samples, the algorithm will be vanilla gradient descent

In [ ]:
prev_uw = weights(3, n, L, X_train, no_of_classes)
prev_ub = biases(3, n, L, y_train, no_of_classes)
 

In [81]:
def Momentum_GD(prev_uw, prev_ub, Weights, bias, hid_layer,
                no_of_neuron, X_train, y_train,learning_rate,
                batch_size, L2_decay, loss_fu, acti_fun):
  
  beta = 0.9
  no_of_classes = len(np.unique(y_train))
  L = hid_layer
  n = no_of_neuron
  
  ## initialize the gradients of weights and biases
  dW = weights(3, n, L, X_train, no_of_classes)
  dB = biases(3, n, L, y_train, no_of_classes)
  ## initialize the loss
  loss = 0
  num_points_seen = 0

  for x,y in zip(X_train, y_train):

    #x = np.float128(x)

    ## Forward propagation
    a_out, h_out, y_dash = forward_propagation(Weights, bias, x, L, acti_fun)
    
    # if loss_fu == 'cross_entropy':
    #   loss_iter = cross_entropy_loss(y_dash, y, X_train)  # for cross entropy loss
    # else:
    #   loss_iter = MSE_loss(y_dash, y_train, X_train)  # for MSE loss use this
    
    # loss += loss_iter

    ## Backward Propagation
    grad_W, grad_b = backward_propagation(a_out, h_out, y_train, y_dash, Weights,
                                          hid_layer, loss_fu, acti_fun, L2_decay)

    ## Adding the gradients of weights and biases
    dW = [dW[i] + grad_W[i] for i in range(len(dW))]
    dB = [dB[i] + grad_b[i] for i in range(len(dB))]

    num_points_seen +=1

    if num_points_seen%batch_size==0:

      ## Add L2 regularization penalty to gradient
      dW = [dW[i] + 2*L2_decay*Weights[i]/len(X_train) for i in range(len(dW))]

      ## momentum based wight updates
      uw = [prev_uw[i]*beta + dW[i]*learning_rate for i in range(len(dW))]
      ub = [prev_ub[i]*beta + dB[i]*learning_rate for i in range(len(dB))]
      
      ## Weights and biases updates
      # Weights updates
      Weights = [Weights[i] - uw[i] for i in range(len(uw))]

      # Biases updates
      bias = [bias[i] - ub[i] for i in range(len(ub))]

      # assign present to the history 
      prev_uw = uw
      prev_ub = ub

      dW = weights(3, n, L, X_train, no_of_classes)
      dB = biases(3, n, L, y_train, no_of_classes)
  
   # Training loss of an epoch
  loss  = model_loss(X_train, y_train, Weights, bias, hid_layer, loss_fu,
                     L2_decay, X_train, X_validation, acti_fun)
  
    # Adding L2 regularization loss
  for i in range(len(Weights)):
    loss = loss + 0.5*L2_decay*np.sum(Weights[i]**2)/len(X_train)

  #print(epoch, loss)

  return  Weights, bias, loss

In [ ]:
Weights, bias = Momentum_GD(Weights, bias, hid_layer, X_train, y_train, 0.9, 0.0001, 25, 0.0005, loss_fu, acti_fun)


0 0.5466685496257547951


In [ ]:
Model_accuracy(X_validation, y_validation, Weights, bias, hid_layer)

53.15

#Nesterov Accelerated Gradient Descent - MiniBatch
if batch_size = 1, the algorithm will be stochastic gradient descent. if batch_size = Number of samples, the algorithm will be vanilla gradient descent

In [55]:
prev_vw = weights(3, 32, 3, X_train, 10)
prev_vb = biases(3, 32, 3, y_train, 10)

In [80]:
def NAG(prev_vw, prev_vb, Weights, bias, hid_layer,
        no_of_neuron, X_train, y_train,learning_rate,
        batch_size, L2_decay, loss_fu, acti_fun):
  
  beta = 0.9
  no_of_classes = len(np.unique(y_train))
  L = hid_layer
  n = no_of_neuron
  
  ## initialize the gradients of weights and biases
  dW = weights(3, n, L, X_train, no_of_classes)
  dB = biases(3, n, L, y_train, no_of_classes)
  ## initialize the loss
  loss = 0
  num_points_seen = 0

  # do partial updates
  v_w = [beta*prev_vw[i] for i in range(len(prev_vw))]
  v_b = [beta*prev_vb[i] for i in range(len(prev_vb))]

  for x, y in zip(X_train, y_train):

    #x = np.float128(x)

    ## Forward propagation
    Weights = [Weights[i]-v_w[i] for i in range(len(Weights))]
    bias    = [bias[i]-v_b[i] for i in range(len(bias))]
    a_out, h_out, y_dash = forward_propagation(Weights, bias, x, L, acti_fun)
    
    # #loss_iter = MSE_loss(y_dash, y_train, X_train)  # for MSE loss use this
    # loss_iter = cross_entropy_loss(y_dash, y, X_train)  # for cross entropy use this
    # loss += loss_iter

    ## Backward Propagation
    grad_W, grad_b = backward_propagation(a_out, h_out, y_train, y_dash, Weights, 
                                          hid_layer, loss_fu, acti_fun, L2_decay)

    ## Look Ahead
    ## Adding the gradients of weights and biases
    dW = [dW[i] + grad_W[i] for i in range(len(dW))]
    dB = [dB[i] + grad_b[i] for i in range(len(dB))]

    num_points_seen +=1

    if num_points_seen%batch_size==0:

      ## Add L2 regularization penalty to gradient
      dW = [dW[i] + 2*L2_decay*Weights[i]/len(X_train) for i in range(len(dW))]

      ## momentum based wight updates
      vw = [prev_vw[i]*beta + dW[i]*learning_rate for i in range(len(dW))]
      vb = [prev_vb[i]*beta + dB[i]*learning_rate for i in range(len(dB))]

      ## Weights and biases updates
      # Weights updates
      Weights = [Weights[i] - vw[i] for i in range(len(vw))]

      # Biases updates
      bias = [bias[i] - vb[i] for i in range(len(vb))]

      # assign present to the history 
      prev_uw = vw
      prev_ub = vb

      dW = weights(3, n, L, X_train, no_of_classes)
      dB = biases(3, n, L, y_train, no_of_classes)
  
   # Training loss of an epoch
  loss  = model_loss(X_train, y_train, Weights, bias, hid_layer, loss_fu,
                     L2_decay, X_train, X_validation, acti_fun)

  # Adding L2 regularization loss
  for i in range(len(Weights)):
    loss = loss + 0.5*L2_decay*np.sum(Weights[i]**2)/len(X_train)

  #print(epoch, loss)

  return  Weights, bias, loss

In [63]:
Weights, bias, loss = NAG(Weights, bias, 3, 32, X_train, y_train,
                    0.0001, 25, 0, 'cross_entropy', 'sigmoid')
  

#Adaptive Gradient(AdaGrad) based Gradient Descent- Minibatch
if batch_size = 1, the algorithm will be stochastic gradient descent. if batch_size = Number of samples, the algorithm will be vanilla gradient descent

In [ ]:
# v_w = weights(3, n, L, X_train, no_of_classes)
# v_b = biases(3, n, L, y_train, no_of_classes)


In [82]:
def AdaGrad(Weights, bias, hid_layer, no_of_neuron, X_train, y_train, eps,
            learning_rate, batch_size, L2_decay, loss_fu, acti_fun):
  
  no_of_classes = len(np.unique(y_train))
  L = hid_layer
  n = no_of_neuron
  
  ## initialize the gradients of weights and biases
  dW = weights(3, n, L, X_train, no_of_classes)
  dB = biases(3, n, L, y_train, no_of_classes)

  ## initialize the loss
  loss = 0
  num_points_seen = 0

  for x, y in zip(X_train, y_train):

    #x = np.float128(x)

    ## Forward propagation
    a_out, h_out, y_dash = forward_propagation(Weights, bias, x, L, acti_fun)
    
    # #loss_iter = MSE_loss(y_dash, y_train, X_train)  # for MSE loss use this
    # loss_iter = cross_entropy_loss(y_dash, y, X_train)  # for cross entropy use this
    # loss += loss_iter

    ## Backward Propagation
    grad_W, grad_b = backward_propagation(a_out, h_out, y_train, y_dash, Weights,
                                          hid_layer, loss_fu, acti_fun, L2_decay)

    dW = [dW[i] + grad_W[i] for i in range(len(dW))]
    dB = [dB[i] + grad_b[i] for i in range(len(dB))]

    num_points_seen +=1

    if num_points_seen%batch_size==0:

      ## Add L2 regularization penalty to gradient
      dW = [dW[i] + 2*L2_decay*Weights[i]/len(X_train) for i in range(len(dW))]

      #compute intermediate values
      v_w = [v_w[i] + dW[i]**2 for i in range(len(grad_W))]
      v_b = [v_b[i] + dB[i]**2 for i in range(len(grad_b))]

      # Weights updates
      Weights = [Weights[i] - learning_rate*dW[i]/(np.sqrt(v_w[i])+eps) for i in range(len(Weights))]

      # Biases updates
      bias = [bias[i] - learning_rate*dB[i]/(np.sqrt(v_b[i])+eps) for i in range(len(bias))]

      dW = weights(3, n, L, X_train, no_of_classes)
      dB = biases(3, n, L, y_train, no_of_classes)
  
   # Training loss of an epoch
  loss  = model_loss(X_train, y_train, Weights, bias, hid_layer, loss_fu,
                     L2_decay, X_train, X_validation, acti_fun)

  # Adding L2 regularization loss
  for i in range(len(Weights)):
    loss = loss + 0.5*L2_decay*np.sum(Weights[i]**2)/len(X_train)

  #print(epoch, loss)


  return  Weights, bias, loss

In [ ]:
Weights, bias = AdaGrad(Weights, bias, hid_layer, no_of_neuron, X_train, y_train, 1e-8,
                        0.0001, 25, L2_decay, loss_fu, acti_fun)
  

#Root Mean Squared Propagation(RMSProp) Gradient Descent - MiniBatch
if batch_size = 1, the algorithm will be stochastic gradient descent. if batch_size = Number of samples, the algorithm will be vanilla gradient descent

In [ ]:
# v_w = weights(3, n, L, X_train, no_of_classes)
# v_b = biases(3, n, L, y_train, no_of_classes)


In [83]:
def RMSProp(v_w, v_b, Weights, bias, hid_layer, no_of_neuron,
            X_train, y_train, learning_rate, batch_size,
            L2_decay,loss_fu, acti_fun):
  
  eps = 1e-10
  beta = 0.9
  no_of_classes = len(np.unique(y_train))
  L = hid_layer
  n = no_of_neuron
  
  ## initialize the gradients of weights and biases
  dW = weights(3, n, L, X_train, no_of_classes)
  dB = biases(3, n, L, y_train, no_of_classes)

  ## initialize the loss
  loss = 0
  num_points_seen = 0

  for x, y in zip(X_train, y_train):

    #x = np.float128(x)

    ## Forward propagation
    a_out, h_out, y_dash = forward_propagation(Weights, bias, x, L, acti_fun)
    
    # #loss_iter = MSE_loss(y_dash, y_train, X_train)  # for MSE loss use this
    # loss_iter = cross_entropy_loss(y_dash, y, X_train)  # for cross entropy use this
    # loss += loss_iter

    ## Backward Propagation
    grad_W, grad_b = backward_propagation(a_out, h_out, y_train, y_dash, Weights,
                                          hid_layer, loss_fu, acti_fun, L2_decay)
    
    dW = [dW[i] + grad_W[i] for i in range(len(dW))]
    dB = [dB[i] + grad_b[i] for i in range(len(dB))]

    num_points_seen +=1

    if num_points_seen%batch_size==0:

      ## Add L2 regularization penalty to gradient
      dW = [dW[i] + 2*L2_decay*Weights[i]/len(X_train) for i in range(len(dW))]

      #compute intermediate values
      v_w = [beta*v_w[i] + (1-beta)*(dW[i]**2) for i in range(len(grad_W))]
      v_b = [beta*v_b[i] + (1-beta)*(dB[i]**2) for i in range(len(grad_b))]

      ## Weights and biases updates
      # Weights updates
      Weights = [Weights[i] - learning_rate*dW[i]/(np.sqrt(v_w[i])+eps) for i in range(len(Weights))]

      # Biases updates
      bias = [bias[i] - learning_rate*dB[i]/(np.sqrt(v_b[i])+eps) for i in range(len(bias))]

      dW = weights(3, n, L, X_train, no_of_classes)
      dB = biases(3, n, L, y_train, no_of_classes)
  
   # Training loss of an epoch
  loss  = model_loss(X_train, y_train, Weights, bias, hid_layer, loss_fu,
                     L2_decay, X_train, X_validation, acti_fun)

  # Adding L2 regularization loss
  for i in range(len(Weights)):
    loss = loss + 0.5*L2_decay*np.sum(Weights[i]**2)/len(X_train)

  #print(epoch, loss)


  return  Weights, bias, loss

In [ ]:
## MiniBatch RMSProp
%%time
Weights, bias = RMSProp(Weights, bias, hid_layer, no_of_neuron, X_train, y_train,
                        learning_rate, batch_size, L2_decay,loss_fu, acti_fun)


0 2.3039213502909686143
CPU times: user 40.5 s, sys: 79.4 ms, total: 40.6 s
Wall time: 49.5 s


#Adaptive Delta(AdaDelta) gradient descent - Minibatch
if batch_size = 1, the algorithm will be stochastic gradient descent. if batch_size = Number of samples, the algorithm will be vanilla gradient descent

In [ ]:
# u_w = weights(3, n, L, X_train, no_of_classes)
# u_b = biases(3, n, L, y_train, no_of_classes)

# v_w = weights(3, n, L, X_train, no_of_classes)
# v_b = biases(3, n, L, y_train, no_of_classes)


In [84]:
def AdaDelta(u_w, u_b, v_w, v_b, Weights, bias, hid_layer,
             no_of_neuron, X_train, y_train, eps, batch_size,
             beta, L2_decay, loss_fu, acti_fun):
  
  no_of_classes = len(np.unique(y_train))
  L, n = hid_layer, no_of_neuron
  
  

  ## initialize the gradients of weights and biases
  dW = weights(3, n, L, X_train, no_of_classes)
  dB = biases(3, n, L, y_train, no_of_classes)

  ## initialize the loss
  loss = 0
  num_points_seen = 0

  for x, y in zip(X_train, y_train):

    #x = np.float128(x)

    ## Forward propagation
    a_out, h_out, y_dash = forward_propagation(Weights, bias, x, L, acti_fun)
    
    # #loss_iter = MSE_loss(y_dash, y_train, X_train)  # for MSE loss use this
    # loss_iter = cross_entropy_loss(y_dash, y, X_train)  # for cross entropy use this
    # loss += loss_iter

    ## Backward Propagation
    grad_W, grad_b = backward_propagation(a_out, h_out, y_train, y_dash, Weights,
                                          hid_layer, loss_fu, acti_fun, L2_decay)
    
    dW = [dW[i] + grad_W[i] for i in range(len(dW))]
    dB = [dB[i] + grad_b[i] for i in range(len(dB))]

    num_points_seen +=1

    if num_points_seen%batch_size==0:

        ## Add L2 regularization penalty to gradient
      dW = [dW[i] + 2*L2_decay*Weights[i]/len(X_train) for i in range(len(dW))]

      #compute intermediate values
      v_w = [beta*v_w[i] + (1-beta)*(dW[i]**2) for i in range(len(grad_W))]
      v_b = [beta*v_b[i] + (1-beta)*(dB[i]**2) for i in range(len(grad_b))]

      del_w = [dW[i]*np.sqrt(u_w[i]+eps)/(np.sqrt(v_w[i]+eps)) for i in range(len(dW))]
      del_b = [dB[i]*np.sqrt(u_b[i]+eps)/(np.sqrt(v_b[i]+eps)) for i in range(len(dB))]

      u_w = [beta*u_w[i] + (1-beta)*del_w[i]**2 for i in range(len(u_w))]
      u_b = [beta*u_b[i] + (1-beta)*del_b[i]**2 for i in range(len(u_b))]

      # Weights updates
      Weights = [Weights[i] - del_w[i] for i in range(len(del_w))]

      # Biases updates
      bias = [bias[i] - del_b[i] for i in range(len(del_b))]

      dW = weights(3, n, L, X_train, no_of_classes)
      dB = biases(3, n, L, y_train, no_of_classes)
  
   # Training loss of an epoch
  loss  = model_loss(X_train, y_train, Weights, bias, hid_layer, loss_fu,
                     L2_decay, X_train, X_validation, acti_fun)

  # Adding L2 regularization loss
  for i in range(len(Weights)):
    loss = loss + 0.5*L2_decay*np.sum(Weights[i]**2)/len(X_train)

  #print(epoch, loss)


  return  Weights, bias, loss

In [ ]:
%%time
## MiniBatch AdaDelta
Weights, bias = AdaDelta(Weights, bias, hid_layer, no_of_neuron, X_train, y_train, 1E-8, 25, 0.9, 0.0005, loss_fu, acti_fun)
  

0 2.3038936342067151898
CPU times: user 38.6 s, sys: 64 ms, total: 38.7 s
Wall time: 41.6 s


#Adaptive moments(Adam) Gradient Descent- MiniBatch
if batch_size = 1, the algorithm will be stochastic gradient descent. if batch_size = Number of samples, the algorithm will be vanilla gradient descent

In [ ]:
# eps = 1e-10
#   beta1 = 0.9
#   beta2 = 0.999
  
#   m_w = weights(3, n, L, X_train, no_of_classes)
#   m_b = biases(3, n, L, y_train, no_of_classes)

#   v_w = weights(3, n, L, X_train, no_of_classes)
#   v_b = biases(3, n, L, y_train, no_of_classes)

In [87]:
def Adam(eps, epoch, beta1, beta2, m_w, m_b, v_w, v_b, Weights,
         bias, hid_layer, no_of_neuron, X_train, y_train, 
         batch_size, learning_rate, L2_decay, loss_fu, acti_fun):
  
  no_of_classes = len(np.unique(y_train))
  L, n = hid_layer, no_of_neuron
  ## initialize the gradients of weights and biases
  dW = weights(3, n, L, X_train, no_of_classes)
  dB = biases(3, n, L, y_train, no_of_classes)

  ## initialize the loss
  loss = 0
  num_points_seen = 0

  for x, y in zip(X_train, y_train):

    #x = np.float128(x)

    ## Forward propagation
    a_out, h_out, y_dash = forward_propagation(Weights, bias, x, L, acti_fun)
    
    # #loss_iter = MSE_loss(y_dash, y_train, X_train)  # for MSE loss use this
    # loss_iter = cross_entropy_loss(y_dash, y, X_train)  # for cross entropy use this
    # loss += loss_iter

    ## Backward Propagation
    grad_W, grad_b = backward_propagation(a_out, h_out, y_train, y_dash, Weights,
                                          hid_layer, loss_fu, acti_fun, L2_decay)
    
    dW = [dW[i] + grad_W[i] for i in range(len(dW))]
    dB = [dB[i] + grad_b[i] for i in range(len(dB))]

    num_points_seen +=1

    if num_points_seen%batch_size==0:

      ## Add L2 regularization penalty to gradient
      dW = [dW[i] + 2*L2_decay*Weights[i]/len(X_train) for i in range(len(dW))]

      #compute intermediate values
      m_w = [beta1*m_w[i] + (1-beta1)*dW[i] for i in range(len(m_w))]
      m_b = [beta1*m_b[i] + (1-beta1)*dB[i] for i in range(len(m_b))]

      v_w = [beta2*v_w[i] + (1-beta2)*(dW[i]**2) for i in range(len(dW))]
      v_b = [beta2*v_b[i] + (1-beta2)*(dB[i]**2) for i in range(len(dB))]

      m_w_hat = [m_w[i]/(1-np.power(beta1,epoch+1)) for i in range(len(m_w))]
      m_b_hat = [m_b[i]/(1-np.power(beta1,epoch+1)) for i in range(len(m_b))]

      v_w_hat = [v_w[i]/(1-np.power(beta2,epoch+1)) for i in range(len(v_w))]
      v_b_hat = [v_b[i]/(1-np.power(beta2,epoch+1)) for i in range(len(v_b))]

      # Weights updates
      Weights = [Weights[i] - learning_rate*m_w_hat[i]/(np.sqrt(v_w_hat[i])+eps) for i in range(len(Weights))]

      # Biases updates
      bias = [bias[i] - learning_rate*m_b_hat[i]/(np.sqrt(v_b_hat[i])+eps) for i in range(len(Weights))]

      dW = weights(3, n, L, X_train, no_of_classes)
      dB = biases(3, n, L, y_train, no_of_classes)

   # Training loss of an epoch
  loss  = model_loss(X_train, y_train, Weights, bias, hid_layer, loss_fu,
                     L2_decay, X_train, X_validation, acti_fun)

    # Adding L2 regularization loss
  for i in range(len(Weights)):
    loss = loss + 0.5*L2_decay*np.sum(Weights[i]**2)/len(X_train)

  #print(epoch, loss)

  return Weights, bias, loss


In [ ]:
Weights, bias = Adam(Weights, bias, hid_layer, no_of_neuron, X_train, y_train, batch_size, learning_rate, L2_decay, loss_fu, acti_fun)

0 1.4280302102562264343


#NAG + Adam = NAdam Gradient descent - MiniBatch
if batch_size = 1, the algorithm will be stochastic gradient descent. if batch_size = Number of samples, the algorithm will be vanilla gradient descent

In [ ]:
# eps = 1e-10
# beta1 = 0.9
# beta2 = 0.999

# m_w = weights(3, n, L, X_train, no_of_classes)
# m_b = biases(3, n, L, y_train, no_of_classes)

# v_w = weights(3, n, L, X_train, no_of_classes)
# v_b = biases(3, n, L, y_train, no_of_classes)


In [89]:
def NAdam(eps, epoch, beta1, beta2, m_w, m_b, v_w, v_b, Weights, bias,
          hid_layer, no_of_neuron, X_train, y_train, batch_size,
          learning_rate,L2_decay, loss_fu, acti_fun):
  
  no_of_classes = len(np.unique(y_train))
  L, n = hid_layer, no_of_neuron
  ## initialize the gradients of weights and biases
  dW = weights(3, n, L, X_train, no_of_classes)
  dB = biases(3, n, L, y_train, no_of_classes)

  ## initialize the loss
  loss = 0
  num_points_seen = 0

  for x, y in zip(X_train, y_train):

    #x = np.float128(x)

    ## Forward propagation
    a_out, h_out, y_dash = forward_propagation(Weights, bias, x, L, acti_fun)
    
    # #loss_iter = MSE_loss(y_dash, y_train, X_train)  # for MSE loss use this
    # loss_iter = cross_entropy_loss(y_dash, y, X_train)  # for cross entropy use this
    # loss += loss_iter

    ## Backward Propagation
    grad_W, grad_b = backward_propagation(a_out, h_out, y_train, y_dash, Weights,
                                          hid_layer, loss_fu, acti_fun, L2_decay)
    
    dW = [dW[i] + grad_W[i] for i in range(len(dW))]
    dB = [dB[i] + grad_b[i] for i in range(len(dB))]

    num_points_seen +=1

    if num_points_seen%batch_size==0:

      ## Add L2 regularization penalty to gradient
      dW = [dW[i] + 2*L2_decay*Weights[i]/len(X_train) for i in range(len(dW))]

      #compute intermediate values
      m_w = [beta1*m_w[i] + (1-beta1)*dW[i] for i in range(len(m_w))]
      m_b = [beta1*m_b[i] + (1-beta1)*dB[i] for i in range(len(m_b))]

      v_w = [beta2*v_w[i] + (1-beta2)*(dW[i]**2) for i in range(len(dW))]
      v_b = [beta2*v_b[i] + (1-beta2)*(dB[i]**2) for i in range(len(dB))]

      m_w_hat = [m_w[i]/(1-np.power(beta1,epoch+1)) for i in range(len(m_w))]
      m_b_hat = [m_b[i]/(1-np.power(beta1,epoch+1)) for i in range(len(m_b))]

      v_w_hat = [v_w[i]/(1-np.power(beta2,epoch+1)) for i in range(len(v_w))]
      v_b_hat = [v_b[i]/(1-np.power(beta2,epoch+1)) for i in range(len(v_b))]

      # Weights updates
      Weights = [Weights[i] - (learning_rate/np.sqrt(v_w_hat[i]+eps))*(beta1*m_w_hat[i]+(1-beta1)*dW[i]/(1-beta1**(epoch+1))) for i in range(len(Weights))]

      # Biases updates
      bias = [bias[i] - (learning_rate/np.sqrt(v_b_hat[i]+eps))*(beta1*m_b_hat[i]+(1-beta1)*dB[i]/(1-beta1**(epoch+1))) for i in range(len(Weights))]

      dW = weights(3, n, L, X_train, no_of_classes)
      dB = biases(3, n, L, y_train, no_of_classes)
  
   # Training loss of an epoch
  loss  = model_loss(X_train, y_train, Weights, bias, hid_layer, loss_fu,
                     L2_decay, X_train, X_validation, acti_fun)

    # Adding L2 regularization loss
  for i in range(len(Weights)):
    loss = loss + 0.5*L2_decay*np.sum(Weights[i]**2)/len(X_train)

  #print(epoch, loss)

  return Weights, bias, loss


In [ ]:
Weights, bias = NAdam(Weights, bias, hid_layer, no_of_neuron, X_train, y_train,
                      batch_size, learning_rate, L2_decay, loss_fu, acti_fun)

# Question-4

In [ ]:
# sweep_config = {"name": "complete-sweep", "method": "grid"}   # may use random search if computation power is less
# sweep_config["metric"] = {"name": "loss", "goal": "minimize"}

# parameters_dict = {
#               "max_epochs": {"values": [5, 10]},
#               "number_of_hidden_layer": {"values": [3, 4, 5]},  
#               "size_hidden_layer": {"values": [32, 64, 128]},           
#               "learning_rate": {"values": [1e-3, 1e-4]},
#               "optimizer": {"values": ["sgd","momentum","nesterov","rmsprop","adam","nadam"]},
#               "batch_size": {"values": [16, 32, 64]}, 
#               "weight_init": {"values": ["random", "xavier"]} ,
#               "activation": {"values": ["Sigmoid", "tanh", "ReLu"]}, 
#               "loss": {"values": ["CrossEntropy", "SquaredError"]}, 
#                 }
# sweep_config["parameters"] = parameters_dict


In [ ]:
# def train_NN(config=sweep_config):
#       with wandb.init(config = config):
#         config = wandb.init().config
#         wandb.run.name = "e_{}_nhl_{}_shl_{}_lr_{}_opt_{}_bs_{}_init_{}_ac_{}_loss_{}".format(config.max_epochs,
#                                                                       config.number_of_hidden_layer,
#                                                                       config.size_hidden_layer,
#                                                                       config.learning_rate,
#                                                                       config.optimizer,
#                                                                       config.batch_size,
#                                                                       config.weight_init,
#                                                                       config.activation,
#                                                                       config.loss)
        

        

In [91]:
def train_NN():
  
  # default values
  config_defaults = {
        'max_epochs': 5,
        'batch_size': 16,
        'learning_rate': 1e-3,
        'acti_fun': 'sigmoid',
        'optimizer': 'sgd',
        'weight_init': 'random',
        'L2_decay': 0,
        'no_of_neuron': 16,
        'hid_layer': 3,
        'loss_fu':'cross_entropy'
    }
  
  # initialize wandb
  wandb.init(config=config_defaults)

  # config is a data structure that holds hyperparameters and inputs
  config = wandb.config

  # Local variables, values obtained from wandb config
  no_of_neuron = config.no_of_neuron
  hid_layer = config.hid_layer
  weight_init = config.weight_init
  max_epochs = config.max_epochs
  batch_size = config.batch_size
  learning_rate = config.learning_rate
  acti_fun = config.acti_fun
  L2_decay = config.L2_decay
  optimizer = config.optimizer
  loss_fu = config.loss_fu

  wandb.run.name  = "e_{}_nhl_{}_shl_{}_lr_{}_opt_{}_bs_{}_W_{}_af_{}_loss_{}".format(max_epochs,
                                                                              hid_layer,
                                                                              no_of_neuron,
                                                                              learning_rate,
                                                                              optimizer,
                                                                              batch_size,
                                                                              weight_init,
                                                                              acti_fun,
                                                                              loss_fu)
                                                                                  
  
  print(wandb.run.name )

  no_of_classes = len(np.unique(y_train))
  Weights = weights(weight_init, no_of_neuron, hid_layer, X_train, no_of_classes)
  bias = biases(weight_init, no_of_neuron, hid_layer, y_train, no_of_classes)
  eps = 1e-10
  beta1 = 0.9
  beta2 = 0.999
  L, n = hid_layer, no_of_neuron
  
  prev_uw = weights(3, n, L, X_train, no_of_classes)
  prev_ub = biases(3, n, L, y_train, no_of_classes)
 
  prev_vw = weights(3, n, L, X_train, no_of_classes)
  prev_vb = biases(3, n, L, y_train, no_of_classes)
  
  m_w = weights(3, n, L, X_train, no_of_classes)
  m_b = biases(3, n, L, y_train, no_of_classes)

  v_w = weights(3, n, L, X_train, no_of_classes)
  v_b = biases(3, n, L, y_train, no_of_classes)

  for epoch in range(max_epochs):

    if optimizer == 'sgd':
      Weights, bias, loss = gradient_descent(learning_rate, Weights, bias, hid_layer, no_of_neuron,
                     y_train, X_train, batch_size, L2_decay, loss_fu, acti_fun)
    elif optimizer == 'momentum':
      Weights, bias, loss = Momentum_GD(prev_uw, prev_ub, Weights, bias, hid_layer,
                                        no_of_neuron, X_train, y_train,learning_rate,
                                        batch_size, L2_decay, loss_fu, acti_fun)
    elif optimizer == 'nesterov':
      Weights, bias, loss = NAG(prev_vw, prev_vb, Weights, bias, hid_layer,
                                no_of_neuron, X_train, y_train,learning_rate,
                                batch_size, L2_decay, loss_fu, acti_fun)
    elif optimizer == 'rmsprop':
      Weights, bias, loss = RMSProp(v_w, v_b, Weights, bias, hid_layer, no_of_neuron,
                                    X_train, y_train, learning_rate, batch_size,
                                    L2_decay,loss_fu, acti_fun)
    elif optimizer == 'adam':
      Weights, bias, loss = Adam(eps, beta1, beta2, m_w, m_b, v_w, v_b, Weights,
                                  bias, hid_layer, no_of_neuron, X_train, y_train, 
                                  batch_size, learning_rate, L2_decay, loss_fu, acti_fun)
    elif optimizer == 'nadam':   
      Weights, bias, loss = NAdam(eps, beta1, beta2, m_w, m_b, v_w, v_b, Weights, bias,
                                  hid_layer, no_of_neuron, X_train, y_train, batch_size,
                                  learning_rate,L2_decay, loss_fu, acti_fun)

    print(epoch, loss)

  
    val_accuracy  = Model_accuracy(X_validation, y_validation, Weights, bias, hid_layer, acti_fun)
    train_accuracy = Model_accuracy(X_test, y_test, Weights, bias, hid_layer, acti_fun)
    val_loss      = model_loss(X_validation, y_validation, Weights, bias, hid_layer, loss_fu, L2_decay, X_train, X_validation, acti_fun)
    train_loss     = model_loss(X_train, y_train, Weights, bias, hid_layer, loss_fu, L2_decay, X_train, X_validation, acti_fun)
    
  
    wandb.log({"validation accuracy": val_accuracy, "train accuracy": train_accuracy, "validation loss": val_loss, "train loss": train_loss, 'epoch': epoch})
    
  wandb.run.name 
  wandb.run.save()
  wandb.run.finish()

  return Weights, bias, loss





#W&B Sweep

In this cell, we set up the configurations for the various hyperparameters and use the Sweeps feature to find the combination that gives us the highest validation accuracy.


In [69]:
sweep_config = {"name": "cs6910_trial_2", "method": "grid"}   # may use random search if computation power is less
sweep_config["metric"] = {"name": "val_loss", "goal": "minimize"}

parameters_dict = {
              "max_epochs": {"values": [5, 10]},
              "hid_layer": {"values": [3, 4, 5]},  
              "no_of_neuron": {"values": [32, 64, 128]},           
              "learning_rate": {"values": [1e-3, 1e-4]},
              "optimizer": {"values": ["sgd","momentum","nesterov","rmsprop","adam","nadam"]},
              "batch_size": {"values": [16, 32, 64]}, 
              "weight_init": {"values": ["random", "xavier"]} ,
              "L2_decay": {"values": [0, 0.0005, 0.5]} ,
              "acti_fun": {"values": ["sigmoid", "tanh", "ReLu"]}, 
              "loss_fu": {"values": ["cross_entropy", "mse"]}, 
                }
sweep_config["parameters"] = parameters_dict

sweep_id = wandb.sweep(sweep_config, entity="am22s020", project="cs6910_trial_3")
wandb.agent(sweep_id, train_NN, count=1)

Create sweep with ID: 1us9ly1l
Sweep URL: https://wandb.ai/am22s020/cs6910_trial_3/sweeps/1us9ly1l


wandb: Agent Starting Run: fmxm41x7 with config:
wandb: 	L2_decay: 0
wandb: 	acti_fun: sigmoid
wandb: 	batch_size: 16
wandb: 	hid_layer: 3
wandb: 	learning_rate: 0.001
wandb: 	loss_fu: cross_entropy
wandb: 	max_epochs: 5
wandb: 	no_of_neuron: 32
wandb: 	optimizer: nesterov
wandb: 	weight_init: random


nhl_3_shl_32_lr_0.001_opt_nesterov_bs_16_W_random_af_sigmoid_loss_cross_entropy
0 2.302585092991647737
1 2.3025850929928566972
2 2.3025850929935658554
3 2.3025850929938316166
4 2.3025850929939351024


epoch,▁▃▅▆█
test accuracy,▁▁▁▁▁
test loss,▁▁▁▁▁
validation accuracy,▁▁▁▁▁
validation loss,▁▁▁▁▁
epoch,4
test accuracy,10.0
test loss,2.30259
validation accuracy,10.01667
validation loss,2.30259
